<a href="https://colab.research.google.com/github/lakhanrajpatlolla/aiml-learning/blob/master/U2W6_19_Aptitude_Classification_Word2Vec_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

### Learning Objectives:

At the end of the experiment, you will be able to:

*  generate vectors using Word2Vec model

In [ ]:
#@title Experiment Walkthrough Video
from IPython.display import HTML

HTML("""<video width="520" height="440" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/Aptitude_classification.mp4">
</video>
""")

## Dataset

Being able to classify the questions will be difficult in natural language processing. The dataset is taken from the TalentSprint aptitude questions.

## Description
This dataset has the following columns:
1. **Category:** Gives the high-level categorization of the question
2. **Sub-Category:** Determines the type of questions
3. **Article:** Gives the article name of the question
4. **Questions:** Questions are listed
5. **Answers:** Contains answers



The dataset, which is considered in the experiment is partially pre-processed using BeautifulSoup and removed punctuations, HTML tags.


### Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2418775" #@param {type:"string"}

In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9959000490" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import re
ipython = get_ipython()

notebook= "U2W6_19_Aptitude_Classification_Word2Vec_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Cleaned_Aptitude_Classification.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/talentsprint1/archives/sc/aiml/experiment_related_data/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    ipython.magic("sx unrar e /content/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getWalkthrough() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook, "feedback_walkthrough":Walkthrough ,
              "feedback_experiments_input" : Comments,
              "feedback_inclass_mentor": Mentor_support}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


def getWalkthrough():
  try:
    if not Walkthrough:
      raise NameError
    else:
      return Walkthrough
  except NameError:
    print ("Please answer Walkthrough Question")
    return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


### Importing required packages





In [4]:
import nltk
import gensim
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Data loading and preparation

Load the aptitude classification dataset containing all the aptitude questions of various sub-categories


In [5]:
data = pd.read_csv("/content/Cleaned_Aptitude_Classification.csv")
data.shape

(4586, 5)

In [6]:
data.head()

,Category,Sub-Category,Article,Questions,Answers
0,Verbal,Misspell words,chapter 1,which of the following is correct,2
1,Quantitative,Time and Distance,Time and Distance - Model 05,rohan leaves point a and reaches point b in 6 ...,2
2,Verbal,Finding Errors,44054,read the sentence to find out whether there is...,2
3,Quantitative,Time and Work,tech mahindra_5th August,4 men can check exam papers in 8 days working ...,2
4,Quantitative,Permutations and Combinations,AX10DEPT01,from 13 persons how many ways of selection of ...,2


Out of 15 sub-categories from the loaded data, choosing two sub-categories (Misspell words, Finding Errors) for this experiment

In [7]:
# Extracting two sub-categories questions
category1_Que = data[data['Sub-Category']== 'Misspell words']['Questions'].values
category2_Que = data[data['Sub-Category']== 'Finding Errors']['Questions'].values

In [9]:
# Printing the sample question from the first chosen Sub-Category
category1_Que[0:5]

array(['which of the following is correct',
       'which of the following is correct',
       'which of the following is correct',
       'which of the following is correct',
       'which of the following is correct'], dtype=object)

### Pre-processing and tokenization

Pre-processing the text and applying tokenization to get vocabulary words of both chosen sub-categories

In [10]:
# Intializing nltk requirements for pre-processing
lemmatizer = WordNetLemmatizer()
stoplist = set(stopwords.words('english'))

In [11]:
# Tokenize the sentence and get vocab words
def Tokenize(AllQuestions):
  pre_processed_words = []
  for each in AllQuestions:
    words = word_tokenize(each)
    words = [lemmatizer.lemmatize(w) for w in words]
    pre_processed_words.extend(words)

  pre_processed_words = set(pre_processed_words)

  pre_processed_words = [word for word in pre_processed_words if word not in stoplist]
  return pre_processed_words

In [12]:
# Calling the above Tokenize function to get vocab words of both sub-categories
category1_words = Tokenize(category1_Que)
category2_words = Tokenize(category2_Que)

# Combining the words of two sub-categories
all_words = category1_words + category2_words
print("Number of valid words after pre-processing:", len(all_words))

Number of valid words after pre-processing: 2163


Load Gensim pretrained model

  * Gensim is an open source Python library for natural language processing. It is developed and is maintained by the Czech natural language processing researcher Radim Rehurek and his company RaRe Technologies.

  * Use gensim to load a word2vec model, pretrained on google news, covering approximately 3 million words and phrases. The vector length is 300 features.

  * Download the google news bin file with the limit 500000 words and save in a binary word2vec format. If **binary = True**, then the data will be saved in binary word2vec format, else it will be saved in plain text.


In [13]:
# Load 300 vectors directly from the file. As the model is in .bin extension, we need to enable default parameter, binary = True
model = gensim.models.KeyedVectors.load_word2vec_format('AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin', binary=True, limit=500000)

In [15]:
# Pre-trained model gives representation of 300 size vector
print("Dimension of the word 'tree': ", len(model['tree']))

Dimension of the word 'tree':  300


### Generate vectors for each word

Words that appear in both the sub-categories will have the same representation but different label, which may lead to less accuracy in classification, Ignoring the words that are intersecting both the chosen sub-categories

In [16]:
# Get vector representation using model for the all the extraced words of two sub-categories
vectors, labels = [], []
for word in all_words:
  try:
    # Ignoring the words that appear in both sub-categories
    if ~(word in category1_words and word in category2_words):
      vectors.append(model[word])
      if word in category1_words:
        labels.append(0)
      else:
        labels.append(1)
  except:
    pass
print("Number of words:", len(labels))
print("Number of dimensions in each vector:", len(vectors[0]))

Number of words: 1743
Number of dimensions in each vector: 300


In [17]:

import numpy as np
np.array(vectors).shape

(1743, 300)

### Split the Data into train and test

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2, random_state=42)

### Fit the model and calculate the accuracy

In [19]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

print("Accuracy of the model:",clf.score(X_test, y_test))

Accuracy of the model: 0.9856733524355301


### Ungraded Exercise:

Take any other two sub-categories and get vector representation using word2vec

In [ ]:
# YOUR CODE HERE

### Please answer the questions below to complete the experiment:




In [20]:
#@title State True or False: Word embeddings capture multiple dimensions of data and are represented as vectors. { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "TRUE" #@param ["","TRUE", "FALSE"]

In [21]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [22]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "Good" #@param {type:"string"}


In [23]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [24]:
#@title  Experiment walkthrough video? { run: "auto", vertical-output: true, display-mode: "form" }
Walkthrough = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [25]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [26]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [27]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 1096
Date of submission:  27 Dec 2024
Time of submission:  15:47:23
View your submissions: https://learn-iiith.talentsprint.com/notebook_submissions
